# IBM Employee Inferential Statstics

![alt text](https://drive.google.com/uc?export=view&id=1RNnstLhQAfK4fMvFs6GG7e9pEdG2APgD)

### From the previous part, the dashboard shows a problem that is to find what factors causing employees to leave this company?
So we start to look it deeper. By doing an Inferential Statistic to create an attrition decision model in order to illustrate the factors that cause resignation problems so that the business can adapt to the problems correctly.



## Import Dataset

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
file_dest = '/content/IBM_dataset.xlsx'
dt = pd.read_excel(file_dest, index_col='EmployeeNumber')
dt.head() #Use only variables that can explain the meaning.

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,JobInvolvement,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
EmployeeNumber,,,,,,,,,,,,,,,,,,,,,,,,,,
1,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,Female,3,Sales Executive,4,Single,5993,8,Yes,0.11,3,1,0,8,1,6,4,0,5
2,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,Male,2,Research Scientist,2,Married,5130,1,No,0.23,4,4,1,10,3,10,7,1,7
4,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,Male,2,Laboratory Technician,3,Single,2090,6,Yes,0.15,3,2,0,7,3,0,0,0,0
5,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,Female,3,Research Scientist,3,Married,2909,1,Yes,0.11,3,3,0,8,3,8,7,3,0
7,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,Male,3,Laboratory Technician,2,Married,3468,9,No,0.12,3,4,1,6,3,2,2,2,2


In [3]:
dt.describe()

,Age,DistanceFromHome,Education,EnvironmentSatisfaction,JobInvolvement,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,9.192517,2.912925,2.721769,2.729932,2.728571,6502.931293,2.693197,0.152095,3.153741,2.712245,0.793878,11.279592,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,8.106864,1.024165,1.093082,0.711561,1.102846,4707.956783,2.498009,0.036599,0.360824,1.081209,0.852077,7.780782,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1009.000000,0.000000,0.110000,3.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2911.000000,1.000000,0.120000,3.000000,2.000000,0.000000,6.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,7.000000,3.000000,3.000000,3.000000,3.000000,4919.000000,2.000000,0.140000,3.000000,3.000000,1.000000,10.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,14.000000,4.000000,4.000000,3.000000,4.000000,8379.000000,4.000000,0.180000,3.000000,4.000000,1.000000,15.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,29.000000,5.000000,4.000000,4.000000,4.000000,19999.000000,9.000000,0.250000,4.000000,4.000000,3.000000,40.000000,4.000000,40.000000,18.000000,15.000000,17.000000


## Data Manipulation

In [4]:
dt.info() #check NULL values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1470 entries, 1 to 2068
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1470 non-null   int64  
 1   Attrition                 1470 non-null   object 
 2   BusinessTravel            1470 non-null   object 
 3   Department                1470 non-null   object 
 4   DistanceFromHome          1470 non-null   int64  
 5   Education                 1470 non-null   int64  
 6   EducationField            1470 non-null   object 
 7   EnvironmentSatisfaction   1470 non-null   int64  
 8   Gender                    1470 non-null   object 
 9   JobInvolvement            1470 non-null   int64  
 10  JobRole                   1470 non-null   object 
 11  JobSatisfaction           1470 non-null   int64  
 12  MaritalStatus             1470 non-null   object 
 13  MonthlyIncome             1470 non-null   int64  
 14  NumCompa

In [5]:
cols_dum = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime', 'Attrition']
dummies=pd.get_dummies(dt[cols_dum], drop_first=True) #dummied those columns
dummies.head()

,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Research & Development,Department_Sales,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Male,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes,Attrition_Yes
EmployeeNumber,,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1
2,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0
4,0,1,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,1,1
5,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0
7,0,1,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0


In [6]:
df = dt.drop(cols_dum, axis=1)
df = pd.concat([df, dummies], axis=1) #merge to a new dataframe
df.head()

,Age,DistanceFromHome,Education,EnvironmentSatisfaction,JobInvolvement,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Research & Development,Department_Sales,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Male,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes,Attrition_Yes
EmployeeNumber,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,41,1,2,2,3,4,5993,8,0.11,3,1,0,8,1,6,4,0,5,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1
2,49,8,1,3,2,2,5130,1,0.23,4,4,1,10,3,10,7,1,7,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0
4,37,2,2,4,2,3,2090,6,0.15,3,2,0,7,3,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,1,1
5,33,3,4,4,3,3,2909,1,0.11,3,3,0,8,3,8,7,3,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0
7,27,2,1,1,3,2,3468,9,0.12,3,4,1,6,3,2,2,2,2,0,1,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0


In [7]:
X = df[list(df.columns)].drop('Attrition_Yes', axis=1)
y = df['Attrition_Yes']
X.head()

,Age,DistanceFromHome,Education,EnvironmentSatisfaction,JobInvolvement,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Research & Development,Department_Sales,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Male,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes
EmployeeNumber,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,41,1,2,2,3,4,5993,8,0.11,3,1,0,8,1,6,4,0,5,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1
2,49,8,1,3,2,2,5130,1,0.23,4,4,1,10,3,10,7,1,7,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
4,37,2,2,4,2,3,2090,6,0.15,3,2,0,7,3,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,1
5,33,3,4,4,3,3,2909,1,0.11,3,3,0,8,3,8,7,3,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1
7,27,2,1,1,3,2,3468,9,0.12,3,4,1,6,3,2,2,2,2,0,1,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0


## Modeling
In this step, I'll use Logistic Regression Model to define an attrition factor because it can be easily interpreted which factor is the most effective to make a decision.

In [8]:
from patsy import dmatrices

In [9]:
y, X = dmatrices("""Attrition_Yes ~ Age + DistanceFromHome + EnvironmentSatisfaction + JobInvolvement + 
       JobSatisfaction + RelationshipSatisfaction + TotalWorkingYears + 
       WorkLifeBalance + YearsInCurrentRole + YearsSinceLastPromotion + 
       YearsWithCurrManager + BusinessTravel_Travel_Frequently + BusinessTravel_Travel_Rarely + 
       EducationField_Marketing + EducationField_Medical + 
       EducationField_Other + Gender_Male + MaritalStatus_Married + 
       MaritalStatus_Single + OverTime_Yes""", 
       data = df, return_type='dataframe') #best model (ฺby Backward Stepwise Selection)

In [10]:
model = sm.Logit(y, X).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.321982
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:          Attrition_Yes   No. Observations:                 1470
Model:                          Logit   Df Residuals:                     1449
Method:                           MLE   Df Model:                           20
Date:                Mon, 22 Jun 2020   Pseudo R-squ.:                  0.2710
Time:                        12:08:48   Log-Likelihood:                -473.31
converged:                       True   LL-Null:                       -649.29
Covariance Type:            nonrobust   LLR p-value:                 1.761e-62
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            2.1774      0.773      2.81

In [11]:
print(np.exp(model.params)) #log-odds (explained coefficient)

Intercept                           8.823034
Age                                 0.976229
DistanceFromHome                    1.037009
EnvironmentSatisfaction             0.675000
JobInvolvement                      0.578948
JobSatisfaction                     0.667561
RelationshipSatisfaction            0.807516
TotalWorkingYears                   0.944617
WorkLifeBalance                     0.737704
YearsInCurrentRole                  0.887232
YearsSinceLastPromotion             1.203492
YearsWithCurrManager                0.908103
BusinessTravel_Travel_Frequently    5.834935
BusinessTravel_Travel_Rarely        2.610072
EducationField_Marketing            1.556294
EducationField_Medical              0.696065
EducationField_Other                0.744269
Gender_Male                         1.512644
MaritalStatus_Married               1.310167
MaritalStatus_Single                3.635488
OverTime_Yes                        5.853293
dtype: float64


## Conclusions & Recommendations

The conclusion for this model are shown as list:
1. These are the factors that causing employees to make a decision to leave this company or not. (Except MaritalStatus_Married and EducationField_Other)
2. The most effective factor that makes employees decide to leave this company is overtime (If employees work overtime, they tend to leave this company about 585% compared to employees who do not work overtime) and the second one is the frequent of the employees traveling.
3. The least effective factor is JobInvolvement (~57% if they have less job responsibility).

To avoid employee attrition problem, the company needs to:
1. The company should take good care of employees and should not let them work more than necessary and checking the reasons of employees who travel frequently.
2. The company must make the environment better, especially Work-Life Balance (WorkLifeBalance 73.7%) and Team working environment (RelationshipSatisfaction 80.7%)